# Route Optimization 

In [1]:
import folium
import openrouteservice as ors
import math
import pandas as pd

In [73]:
dtf = pd.read_csv('data_stores.csv')

In [74]:
dtf

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25595,Starbucks,21401-212072,Rex,Licensed,"141 Nguyễn Huệ, Quận 1, Góc đường Pasteur và L...",Thành Phố Hồ Chí Minh,SG,VN,70000,08 3824 4668,GMT+000000 Asia/Saigon,106.70,10.78
25596,Starbucks,24010-226985,Panorama,Licensed,"SN-44, Tòa Nhà Panorama, 208 Trần Văn Trà, Quận 7",Thành Phố Hồ Chí Minh,SG,VN,70000,08 5413 8292,GMT+000000 Asia/Saigon,106.71,10.72
25597,Starbucks,47608-253804,Rosebank Mall,Licensed,"Cnr Tyrwhitt and Cradock Avenue, Rosebank",Johannesburg,GT,ZA,2194,27873500159,GMT+000000 Africa/Johannesburg,28.04,-26.15
25598,Starbucks,47640-253809,Menlyn Maine,Licensed,"Shop 61B, Central Square, Cnr Aramist & Coroba...",Menlyn,GT,ZA,181,NaN,GMT+000000 Africa/Johannesburg,28.28,-25.79


In [76]:
dfr = dtf[dtf['City']=='Regina']
dfr

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
1906,Starbucks,47245-257147,Save On Foods Regina South,Licensed,4520 Albert Street,Regina,SK,CA,S4S 6B4,13065465739,GMT-06:00 America/Regina,-104.62,50.41
1907,Starbucks,79543-102411,Safeway - Regina #367,Licensed,353 Albert St North,Regina,SK,CA,S4R 3C3,13065438749,GMT-06:00 America/Regina,-104.62,50.48
1908,Starbucks,3937-128705,Safeway - Regina #354,Licensed,3015 Gordon Rd,Regina,SK,CA,S4S 6H7,13065865140,GMT-06:00 America/Regina,-104.62,50.40
1909,Starbucks,6171-149827,Safeway-Regina #397,Licensed,3859 Sherwood Drive,Regina,SK,CA,S4R 4A8,306-949-7488,GMT-06:00 America/Regina,-104.64,50.48
1910,Starbucks,79530-102414,Safeway - Regina #399,Licensed,2223 Victoria Ave East,Regina,SK,CA,S4P 0S4,13067893191,GMT-06:00 America/Regina,-104.55,50.45
1911,Starbucks,19551-193811,Safeway - Regina #301,Licensed,"2931 13th Avenue, Southland Mall",Regina,SK,CA,S4T 1N8,306-522-5453,GMT-06:00 America/Regina,-104.62,50.45
1912,Starbucks,4352-2303,Southland Chapters,Company Owned,2627 Gordon Road,Regina,SK,CA,S4S 6H7,306-569-9766,GMT-06:00 America/Regina,-104.62,50.40
1914,Starbucks,25109-226424,Hilton Doubletree Regina,Licensed,1975 Broad St,Regina,SK,CA,S4P1Y2,3065257589,GMT-06:00 America/Regina,-104.61,50.45


In [79]:
# We take only Fredericton for this experiement, can use other cities or intercities as well but for now let us proceed
city = 'Regina'
dtf=dtf[dtf["City"]==city][ ["City", "Street Address", "Latitude", "Longitude"]].reset_index(drop=True)
dtf=dtf.reset_index().rename(columns={"index":"id", "Latitude":"y", "Longitude":"x"})

print("total", len(dtf))
dtf.head(10)

total 8


,id,City,Street Address,y,x
0,0,Regina,4520 Albert Street,50.41,-104.62
1,1,Regina,353 Albert St North,50.48,-104.62
2,2,Regina,3015 Gordon Rd,50.40,-104.62
3,3,Regina,3859 Sherwood Drive,50.48,-104.64
4,4,Regina,2223 Victoria Ave East,50.45,-104.55
5,5,Regina,"2931 13th Avenue, Southland Mall",50.45,-104.62
6,6,Regina,2627 Gordon Road,50.40,-104.62
7,7,Regina,1975 Broad St,50.45,-104.61


In [60]:
# visualize the points on a map 
# Fetching coordinates
coords = [[row["x"], row["y"]] for index, row in dtf.iterrows()]

m = folium.Map(location=list(reversed([-104.62,50.40])), tiles="cartodbpositron", zoom_start=14)
for coord in coords:
    folium.Marker(location=list(reversed(coord))).add_to(m)
m

In [62]:
# Get Open route service client Id and map the starting point
client = ors.Client(key='Your Key')

# Example starting point (you can set it as needed)
vehicle_start = [-104.62,50.40]

m = folium.Map(location=list(reversed([-104.62,50.40])), tiles="cartodbpositron", zoom_start=14)
for coord in coords:
    folium.Marker(location=list(reversed(coord))).add_to(m)
    
folium.Marker(location=list(reversed(vehicle_start)), icon=folium.Icon(color="red")).add_to(m)

m

We do a simple 2 vehicle optimization with each vehicle being able to take a maximum of 5 tasks.

In [63]:
# We can use 2 trucks for now, this can be changed accordingly 
vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[5]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[5])
    ]
jobs = [ors.optimization.Job(id=index, location=coords, amount=[1]) for index, coords in enumerate(coords)]
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)
line_colors = ['green', 'orange']
for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(m)
m

### We now add service durations to our jobs and give each vehicle an 8-hour shift. One vehicle takes 2 jobs, while the other vehicles picks up the remaining 6. There are more options to add restrictions to jobs/vehicles, check out the remaining options at the VROOM API. Note that as the Vroom API changes options may differ slightly from what is available on openrouteservice and implemented in openrouteservice-py

In [89]:
import folium
import math

# Assuming dtf is your DataFrame and contains 'x' and 'y' columns for coordinates
# Create a sample service time for demonstration (adjust as needed)
service_times = [1, 0.5, 1, 6, 1, 1, 1, 0.25]  # Example service times in hours

# Fetching coordinates and creating the coords list
coords = [
    { 'location': [row["x"], row["y"]], 'service': service_times[index] * 60 * 60 }
    for index, row in dtf.iterrows()
]

# Initialize the map
m = folium.Map(location=list(reversed([-104.62, 50.40])), tiles="cartodbpositron", zoom_start=14)

# Add markers to the map
for job in coords:
    folium.Marker(
        location=list(reversed(job['location'])),
        popup=f"Service time: {job['service'] / 60:.1f} minutes"
    ).add_to(m)

# Define the starting point for vehicles
vehicle_start = [-104.62, 50.40]

# Add the vehicle starting point marker
folium.Marker(location=list(reversed(vehicle_start)), icon=folium.Icon(color="red")).add_to(m)

# Example vehicle setup (adjust as needed)
vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=vehicle_start, end=vehicle_start, time_window=[0, 8*60*60]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=vehicle_start, end=vehicle_start, time_window=[0, 8*60*60])
]

# Prepare jobs for optimization
jobs = [ors.optimization.Job(id=index, **job) for index, job in enumerate(coords)]
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)

# Prepare a list to store the route information
route_info = []

# Add optimized routes to the map
line_colors = ['green', 'orange', 'blue', 'yellow']
for route in optimized['routes']:
    folium.PolyLine(
        locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
        color=line_colors[route['vehicle']]
    ).add_to(m)

    for step in route['steps']:
        if step['type'] == 'job':
            folium.Marker(
                location=list(reversed(step['location'])),
                popup=f"Arrival time: {math.floor(step['arrival'] / (60*60))} hours {math.floor((step['arrival'] % (60*60)) / 60)} minutes",
                icon=folium.Icon(color=line_colors[route['vehicle']])
            ).add_to(m)
            
for route in optimized['routes']:
    vehicle_id = route['vehicle']
    for stop_no, step in enumerate(route['steps']):
        if step['type'] == 'job':
            arrival_time = step['arrival']
            arrival_hours = math.floor(arrival_time / (60 * 60))
            arrival_minutes = math.floor((arrival_time % (60 * 60)) / 60)

            # Append the route information to the list
            route_info.append({
                'Vehicle ID': vehicle_id + 1,
                'X': coords[step['job']]['location'][0],
                'Y': coords[step['job']]['location'][1],
                'Arrival Time': f"{arrival_hours} hours {arrival_minutes} minutes",
                'Stop No': stop_no  # Stop numbers starting from 1
            })
            

# Display the map
m


### Now let us print the route details for the drivers 

In [90]:
dfroute = pd.DataFrame(route_info)
dfroute

,Vehicle ID,X,Y,Arrival Time,Stop No
0,1,-104.62,50.40,0 hours 0 minutes,1
1,1,-104.62,50.41,1 hours 3 minutes,2
2,1,-104.62,50.45,2 hours 12 minutes,3
3,1,-104.61,50.45,3 hours 14 minutes,4
4,1,-104.55,50.45,3 hours 38 minutes,5
5,1,-104.62,50.40,4 hours 49 minutes,6
6,2,-104.64,50.48,0 hours 15 minutes,1
7,2,-104.62,50.48,6 hours 19 minutes,2


In [92]:
# Let us merge the two dataframes 

# Merge the DataFrames based on X and Y coordinates
merged_df = pd.merge(dfroute, dfr, left_on=['X', 'Y'], right_on=['Longitude', 'Latitude'], how='inner')

merged_df

,Vehicle ID,X,Y,Arrival Time,Stop No,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,1,-104.62,50.40,0 hours 0 minutes,1,Starbucks,3937-128705,Safeway - Regina #354,Licensed,3015 Gordon Rd,Regina,SK,CA,S4S 6H7,13065865140,GMT-06:00 America/Regina,-104.62,50.40
1,1,-104.62,50.40,0 hours 0 minutes,1,Starbucks,4352-2303,Southland Chapters,Company Owned,2627 Gordon Road,Regina,SK,CA,S4S 6H7,306-569-9766,GMT-06:00 America/Regina,-104.62,50.40
2,1,-104.62,50.41,1 hours 3 minutes,2,Starbucks,47245-257147,Save On Foods Regina South,Licensed,4520 Albert Street,Regina,SK,CA,S4S 6B4,13065465739,GMT-06:00 America/Regina,-104.62,50.41
3,1,-104.62,50.45,2 hours 12 minutes,3,Starbucks,19551-193811,Safeway - Regina #301,Licensed,"2931 13th Avenue, Southland Mall",Regina,SK,CA,S4T 1N8,306-522-5453,GMT-06:00 America/Regina,-104.62,50.45
4,1,-104.61,50.45,3 hours 14 minutes,4,Starbucks,25109-226424,Hilton Doubletree Regina,Licensed,1975 Broad St,Regina,SK,CA,S4P1Y2,3065257589,GMT-06:00 America/Regina,-104.61,50.45
5,1,-104.55,50.45,3 hours 38 minutes,5,Starbucks,79530-102414,Safeway - Regina #399,Licensed,2223 Victoria Ave East,Regina,SK,CA,S4P 0S4,13067893191,GMT-06:00 America/Regina,-104.55,50.45
6,1,-104.62,50.40,4 hours 49 minutes,6,Starbucks,3937-128705,Safeway - Regina #354,Licensed,3015 Gordon Rd,Regina,SK,CA,S4S 6H7,13065865140,GMT-06:00 America/Regina,-104.62,50.40
7,1,-104.62,50.40,4 hours 49 minutes,6,Starbucks,4352-2303,Southland Chapters,Company Owned,2627 Gordon Road,Regina,SK,CA,S4S 6H7,306-569-9766,GMT-06:00 America/Regina,-104.62,50.40
8,2,-104.64,50.48,0 hours 15 minutes,1,Starbucks,6171-149827,Safeway-Regina #397,Licensed,3859 Sherwood Drive,Regina,SK,CA,S4R 4A8,306-949-7488,GMT-06:00 America/Regina,-104.64,50.48
9,2,-104.62,50.48,6 hours 19 minutes,2,Starbucks,79543-102411,Safeway - Regina #367,Licensed,353 Albert St North,Regina,SK,CA,S4R 3C3,13065438749,GMT-06:00 America/Regina,-104.62,50.48


### There seems to be overlap as two locations have same longitudes and latitudes and so we remove either one to enable the route

In [93]:
# Remove duplicates
merged_df.drop_duplicates(subset=['Vehicle ID', 'X', 'Y'], keep='first', inplace=True)
merged_df

,Vehicle ID,X,Y,Arrival Time,Stop No,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,1,-104.62,50.40,0 hours 0 minutes,1,Starbucks,3937-128705,Safeway - Regina #354,Licensed,3015 Gordon Rd,Regina,SK,CA,S4S 6H7,13065865140,GMT-06:00 America/Regina,-104.62,50.40
2,1,-104.62,50.41,1 hours 3 minutes,2,Starbucks,47245-257147,Save On Foods Regina South,Licensed,4520 Albert Street,Regina,SK,CA,S4S 6B4,13065465739,GMT-06:00 America/Regina,-104.62,50.41
3,1,-104.62,50.45,2 hours 12 minutes,3,Starbucks,19551-193811,Safeway - Regina #301,Licensed,"2931 13th Avenue, Southland Mall",Regina,SK,CA,S4T 1N8,306-522-5453,GMT-06:00 America/Regina,-104.62,50.45
4,1,-104.61,50.45,3 hours 14 minutes,4,Starbucks,25109-226424,Hilton Doubletree Regina,Licensed,1975 Broad St,Regina,SK,CA,S4P1Y2,3065257589,GMT-06:00 America/Regina,-104.61,50.45
5,1,-104.55,50.45,3 hours 38 minutes,5,Starbucks,79530-102414,Safeway - Regina #399,Licensed,2223 Victoria Ave East,Regina,SK,CA,S4P 0S4,13067893191,GMT-06:00 America/Regina,-104.55,50.45
8,2,-104.64,50.48,0 hours 15 minutes,1,Starbucks,6171-149827,Safeway-Regina #397,Licensed,3859 Sherwood Drive,Regina,SK,CA,S4R 4A8,306-949-7488,GMT-06:00 America/Regina,-104.64,50.48
9,2,-104.62,50.48,6 hours 19 minutes,2,Starbucks,79543-102411,Safeway - Regina #367,Licensed,353 Albert St North,Regina,SK,CA,S4R 3C3,13065438749,GMT-06:00 America/Regina,-104.62,50.48


In [96]:
merged_df.drop(['X','Y'],axis=1, inplace=True)
merged_df

,Vehicle ID,Arrival Time,Stop No,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,1,0 hours 0 minutes,1,Starbucks,3937-128705,Safeway - Regina #354,Licensed,3015 Gordon Rd,Regina,SK,CA,S4S 6H7,13065865140,GMT-06:00 America/Regina,-104.62,50.40
2,1,1 hours 3 minutes,2,Starbucks,47245-257147,Save On Foods Regina South,Licensed,4520 Albert Street,Regina,SK,CA,S4S 6B4,13065465739,GMT-06:00 America/Regina,-104.62,50.41
3,1,2 hours 12 minutes,3,Starbucks,19551-193811,Safeway - Regina #301,Licensed,"2931 13th Avenue, Southland Mall",Regina,SK,CA,S4T 1N8,306-522-5453,GMT-06:00 America/Regina,-104.62,50.45
4,1,3 hours 14 minutes,4,Starbucks,25109-226424,Hilton Doubletree Regina,Licensed,1975 Broad St,Regina,SK,CA,S4P1Y2,3065257589,GMT-06:00 America/Regina,-104.61,50.45
5,1,3 hours 38 minutes,5,Starbucks,79530-102414,Safeway - Regina #399,Licensed,2223 Victoria Ave East,Regina,SK,CA,S4P 0S4,13067893191,GMT-06:00 America/Regina,-104.55,50.45
8,2,0 hours 15 minutes,1,Starbucks,6171-149827,Safeway-Regina #397,Licensed,3859 Sherwood Drive,Regina,SK,CA,S4R 4A8,306-949-7488,GMT-06:00 America/Regina,-104.64,50.48
9,2,6 hours 19 minutes,2,Starbucks,79543-102411,Safeway - Regina #367,Licensed,353 Albert St North,Regina,SK,CA,S4R 3C3,13065438749,GMT-06:00 America/Regina,-104.62,50.48


In [102]:
# For vehicle 1 and vehicle 2 to a excel

df1= merged_df[merged_df['Vehicle ID']==1]
df1.to_excel('route_details_for_vehicle1.xlsx',index=False)
df2= merged_df[merged_df['Vehicle ID']==2]
df1.to_excel('route_details_for_vehicle2.xlsx',index=False)
